# **Real Estate Analysis - Resale of Singapore HDB Flats** 
---

## What's This Project About? 
 ### **_You have historical data on the resale prices of government-built flats in Singapore._**
## Your goal is to use this data to help different groups of people: 
## - Potential Buyers/Sellers: To understand what a fair price is for a flat. 
## - Property Analysts: To identify market trends. 
## - Government Agencies: To understand housing affordability and market behavior.
  

## HOUSING MARKET MONITOR - DATA CLEANING 
---

In [1]:
import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings('ignore')

## 1. Load all files

In [2]:
files = [
    "resale-flat-prices-based-on-approval-date-2000-feb-2012.csv",
    "resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv",
    "resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv",
    "ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv",
    
]

In [3]:
dfs = []
for f in files:
    df = pd.read_csv(f, low_memory=False)
    df['source_file'] = f  # for debugging
    dfs.append(df)

## 2. Concatenate everything

In [4]:
df = pd.concat(dfs, ignore_index=True)

In [5]:
print(f"Total raw rows: {len(df):,}")
df.head()

Total raw rows: 628,591


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,source_file,remaining_lease
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,resale-flat-prices-based-on-approval-date-2000...,NaN
1,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0,resale-flat-prices-based-on-approval-date-2000...,NaN
2,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0,resale-flat-prices-based-on-approval-date-2000...,NaN
3,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0,resale-flat-prices-based-on-approval-date-2000...,NaN
4,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0,resale-flat-prices-based-on-approval-date-2000...,NaN


##  3. BASIC CLEANING 

In [6]:
df.columns = df.columns.str.lower()

### Fix month
---

In [7]:
df['month'] = pd.to_datetime(df['month'], format='%Y-%m', errors='coerce')
df = df.dropna(subset=['month'])
df['transaction_year'] = df['month'].dt.year
df['month_str'] = df['month'].dt.strftime('%Y-%m')

### Numerical columns
---

In [8]:
df['floor_area_sqm'] = pd.to_numeric(df['floor_area_sqm'], errors='coerce')
df['resale_price'] = pd.to_numeric(df['resale_price'], errors='coerce')
df['lease_commence_date'] = pd.to_numeric(df['lease_commence_date'], errors='coerce')

## 4. REMAINING LEASE

In [9]:
def parse_remaining_lease(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip().lower()
    if x == '' or x == 'nan':
        return np.nan
    
    years = 0.0
    months = 0.0
    
    if 'year' in x:
        parts = x.split('year')
        years = int(parts[0].strip())
        if len(parts) > 1 and 'month' in parts[1]:
            months_part = parts[1].replace('month', '').replace('s', '').strip()
            if months_part:
                months = int(months_part.split()[0])
    elif x.replace('.','').isdigit():
        years = float(x)
    
    return years + months/12

# Parse where available (2015+)
df['remaining_lease_years'] = df['remaining_lease'].apply(parse_remaining_lease)

# Calculate flat age at sale
df['flat_age_at_sale'] = df['transaction_year'] - df['lease_commence_date']

# Fill missing remaining lease using official 99-year rule
mask = df['remaining_lease_years'].isna()
df.loc[mask, 'remaining_lease_years'] = 99 - df.loc[mask, 'flat_age_at_sale']

# Safety
df['remaining_lease_years'] = df['remaining_lease_years'].clip(0, 99)
df['remaining_lease_rounded'] = df['remaining_lease_years'].round(0).astype(int)

## 5. Clean flat type
---

In [10]:
df['price_per_sqm'] = df['resale_price'] / df['floor_area_sqm']

# Storey midpoint
def get_mid(st):
    if pd.isna(st):
        return np.nan
    a, b = map(int, str(st).split(' TO '))
    return (a + b) // 2 + (1 if (a + b) % 2 else 0)

df['storey_mid'] = df['storey_range'].apply(get_mid)

# Clean flat_type
df['flat_type'] = df['flat_type'].str.upper().str.replace('-', ' ').str.strip()
df['flat_type'] = df['flat_type'].replace({'MULTI GENERATION': 'MULTI-GENERATION'})

##  6. FINAL CLEANUP
---

### Drop messy columns

In [11]:
df = df.drop(columns=['remaining_lease', 'source_file'], errors='ignore')

### Remove obvious junk rows

In [12]:
df = df.dropna(subset=['resale_price', 'floor_area_sqm', 'lease_commence_date'])
df = df[df['resale_price'] > 100000]
df = df[df['floor_area_sqm'] >= 28]

### Sort by date

In [13]:
df = df.sort_values('month').reset_index(drop=True)

### Save in CSV file

In [14]:
df.to_csv("cleandf1.csv", index=False)

### Print file

In [15]:
print("\nCLEANING COMPLETE! Final master file saved.")
print(f"Total clean rows: {len(df):,}")
print(f"Date range: {df['month'].min().date()} → {df['month'].max().date()}")
print("\nSample of remaining lease calculation:")
print(df[['month','lease_commence_date','transaction_year','flat_age_at_sale','remaining_lease_years']].head(10))


CLEANING COMPLETE! Final master file saved.
Total clean rows: 625,599
Date range: 2000-01-01 → 2024-01-01

Sample of remaining lease calculation:
       month  lease_commence_date  transaction_year  flat_age_at_sale  \
0 2000-01-01                 1986              2000                14   
1 2000-01-01                 1979              2000                21   
2 2000-01-01                 1978              2000                22   
3 2000-01-01                 1967              2000                33   
4 2000-01-01                 1967              2000                33   
5 2000-01-01                 1967              2000                33   
6 2000-01-01                 1967              2000                33   
7 2000-01-01                 1979              2000                21   
8 2000-01-01                 1967              2000                33   
9 2000-01-01                 1967              2000                33   

   remaining_lease_years  
0                   85

### further cleaning done in excel (cleandf1-copy1.csv)